# Подготовка данных

In [1]:
import requests
import pandas as pd

res = requests.get('http://10.32.1.107:5300/api/v1/physical_object_types')
physical_objects_types = pd.DataFrame(res.json()).set_index('physical_object_type_id', drop=True)
# physical_objects_types.to_excel('pot.xlsx')
physical_objects_types

name  \
physical_object_type_id                                                      
1                                                       Рекреационная зона   
2                                                            Водный объект   
3                                                             Зеленая зона   
4                                                                Жилой дом   
5                                                           Нежилое здание   
7                                                                 Площадка   
8                                                      Заправочная станция   
9                                                                 Памятник   
10                                          Остановка наземного транспорта   
11                                                                Кладбище   
13                       Распределительный трубопровод для транспортиро...   
14                                                Электрическая подстанция   
18                                                Магистральный газопровод   
19                                                                Парковка   
20                                                   Линия электропередачи   
21                                                          Электростанция   
23                                                                 Зоопарк   
24                                                      Сеть водоотведения   
27                                                      Сеть водоснабжения   
28                                                                    Порт   
29                                                                   Склад   
30                                                 Железнодорожная станция   
31                                                                    Пляж   
33                                                          Ветрогенератор   
34                                                     Гидроэлектростанция   
35                                                  Атомная электростанция   
36                                                        Парк развлечений   
37                                             Сооружение для очистки воды   
38                                                      Водонапорная башня   
39                                                 Водоочистное сооружение   
40                                                 Водозаборное сооружение   
41                                                               Котельная   
42                                          Насосная станция водоснабжения   
43                                                 Промышленная территория   
44                                                                    Река   
45                                                                   Озеро   
46                                                                    Поле   
47                                                       Травяное покрытие   
48                                                                     Лес   
49                                                             Полигон ТБО   

                                                  physical_object_function  
physical_object_type_id                                                     
1                                  {'id': 3, 'name': 'Рекреационная зона'}  
2                                       {'id': 4, 'name': 'Водный объект'}  
3                                        {'id': 2, 'name': 'Зеленая зона'}  
4                                              {'id': 1, 'name': 'Здание'}  
5                                              {'id': 1, 'name': 'Здание'}  
7                                            {'id': 5, 'name': 'Площадка'}  
8                                 {'id': 9, 'name': 'Заправочная станция'}  
9                              {'id': 10, 'name': 'Достопримечательность'}  
10                     

In [2]:
from enum import Enum
class EngineeringObject(Enum):
    POWER_SUPPLY = 'Энергоснабжение'
    HEAT_SUPPLY = 'Теплоснабжение'
    GAS_SUPPLY = 'Газоснабжение'
    WATER_SUPPLY = 'Водоснабжение'
    WATER_DRAINAGE = 'Водоотведение'

In [3]:
ENG_OBJ_TO_INDICATOR = {
    EngineeringObject.POWER_SUPPLY: [14, 20, 21, 33, 34, 35],  # Электрические подстанции, ЛЭП, электростанции и генераторы
    EngineeringObject.HEAT_SUPPLY: [41],                       # Котельная
    EngineeringObject.GAS_SUPPLY: [13, 18],                    # Магистральный газопровод
    EngineeringObject.WATER_SUPPLY: [27, 38, 40, 42],          # Сети водоснабжения, водонапорные башни, водозаборные и насосные станции
    EngineeringObject.WATER_DRAINAGE: [24, 37, 39]             # Сети водоотведения, сооружения для очистки воды, водоочистные сооружения
}



In [4]:
import requests
import geopandas as gpd
import shapely
import json
import pandas as pd

URBAN_API = 'http://10.32.1.107:5300'
PAGE_SIZE = 10_000

def _get_physical_objects(region_id : int, pot_id : int, page : int, page_size : int = PAGE_SIZE):
  res = requests.get(f'{URBAN_API}/api/v1/territory/{region_id}/physical_objects_with_geometry', {
    'physical_object_type_id': pot_id,
    'page': page,
    'page_size': page_size,
  })
  return res.json()

def get_physical_objects(region_id : int, pot_id : int):
  page = 1
  results = []
  while True:
    res_json = _get_physical_objects(region_id, pot_id, page, page_size=PAGE_SIZE)
    results.extend(res_json['results'])
    if res_json['next'] is None:
      break
    page += 1
  
  # Фильтрация объектов с наличием geometry
  results_with_geometry = [result for result in results if 'geometry' in result and result['geometry'] is not None]
  
  # Если нет объектов с геометрией, возвращаем пустой GeoDataFrame
  if not results_with_geometry:
    return gpd.GeoDataFrame(columns=['geometry'])
  
  # Конвертация geometry
  for result in results_with_geometry:
    g = result['geometry']
    result['geometry'] = shapely.from_geojson(json.dumps(g))
  
  return gpd.GeoDataFrame(results_with_geometry).set_geometry('geometry')

def fetch_required_objects(region_id : int, pot_ids : list[int]):
  gdfs = [get_physical_objects(region_id, pot_id) for pot_id in pot_ids]
  return pd.concat(gdfs).set_geometry('geometry').set_crs(4326)

In [5]:
gdfs = {}
for eng_obj, ind_id in ENG_OBJ_TO_INDICATOR.items():
  if len(ind_id) > 0:
    gdf = fetch_required_objects(1, ind_id)
    gdfs[eng_obj] = gdf
gdfs

{<EngineeringObject.POWER_SUPPLY: 'Энергоснабжение'>:                      geometry  physical_object_id  \
 0   POINT (30.50513 59.84176)            759813.0   
 1   POINT (30.50316 59.84188)            759823.0   
 2    POINT (30.42919 60.0376)            759836.0   
 3   POINT (30.49322 59.89584)            759924.0   
 4   POINT (30.26719 60.11068)            759957.0   
 5   POINT (29.84225 59.87058)            759959.0   
 6   POINT (30.07734 59.77243)            759960.0   
 7   POINT (30.47855 59.90406)            759965.0   
 8   POINT (30.09877 59.78206)            759969.0   
 9     POINT (30.49359 59.939)            759973.0   
 10  POINT (30.48904 59.94282)            759984.0   
 11  POINT (30.48905 59.94284)            759990.0   
 12  POINT (30.48398 59.94375)            760001.0   
 
                                  physical_object_type  \
 0   {'physical_object_type_id': 35, 'name': 'Атомн...   
 1   {'physical_object_type_id': 35, 'name': 'Атомн...   
 2   {'physical

In [6]:
def get_engineering_gdf(data_dict: dict) -> gpd.GeoDataFrame:
    combined_gdf = gpd.GeoDataFrame(columns=['type', 'geometry'], crs="EPSG:4326")

    for eng_obj, gdf in data_dict.items():
        gdf = gdf.copy()  
        gdf['type'] = eng_obj.value 
        combined_gdf = pd.concat([combined_gdf, gdf], ignore_index=True)

    combined_gdf = gpd.GeoDataFrame(combined_gdf, geometry='geometry', crs="EPSG:4326")
    return combined_gdf


In [7]:
combined_gdf = get_engineering_gdf(gdfs)
combined_gdf

type                                           geometry  \
0   Энергоснабжение                          POINT (30.50513 59.84176)   
1   Энергоснабжение                          POINT (30.50316 59.84188)   
2   Энергоснабжение                           POINT (30.42919 60.0376)   
3   Энергоснабжение                          POINT (30.49322 59.89584)   
4   Энергоснабжение                          POINT (30.26719 60.11068)   
5   Энергоснабжение                          POINT (29.84225 59.87058)   
6   Энергоснабжение                          POINT (30.07734 59.77243)   
7   Энергоснабжение                          POINT (30.47855 59.90406)   
8   Энергоснабжение                          POINT (30.09877 59.78206)   
9   Энергоснабжение                            POINT (30.49359 59.939)   
10  Энергоснабжение                          POINT (30.48904 59.94282)   
11  Энергоснабжение                          POINT (30.48905 59.94284)   
12  Энергоснабжение                          POINT (30.48398 59.94375)   
13    Водоснабжение                          POINT (30.13191 59.79087)   
14    Водоснабжение                          POINT (30.50604 59.83011)   
15    Водоснабжение                          POINT (30.50604 59.83013)   
16    Водоснабжение                          POINT (30.27754 59.70902)   
17    Водоотведение  POLYGON ((30.06831 59.81601, 30.06829 59.81597...   

    physical_object_id                               physical_object_type  \
0             759813.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
1             759823.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
2             759836.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
3             759924.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
4             759957.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
5             759959.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
6             759960.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
7             759965.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
8             759969.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
9             759973.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
10            759984.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
11            759990.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
12            760001.0  {'physical_object_type_id': 35, 'name': 'Атомн...   
13            760454.0  {'physical_object_type_id': 38, 'name': 'Водон...   
14            760457.0  {'physical_object_type_id': 38, 'name': 'Водон...   
15            760467.0  {'physical_object_type_id': 38, 'name': 'Водон...   
16            760468.0  {'physical_object_type_id': 38, 'name': 'Водон...   
17            760612.0  {'physical_object_type_id': 39, 'name': 'Водоо...   

                               name address osm_id properties  \
0   (Physical object for (unnamed))    None   None         {}   
1   (Physical object for (unnamed))    None   None         {}   
2   (Physical object for (unnamed))    None   None         {}   
3   (Physical object for (unnamed))    None   None         {}   
4   (Physical object for (unnamed))    None   None         {}   
5   (Physical object for (unnamed))    None   None         {}   
6   (Physical object for (unnamed))    None   None         {}   
7   (Physical object for (unnamed))    None   None         {}   
8   (Physical object for (unnamed))    None   None         {}   
9   (Physical object for (unnamed))    None   None         {}   
10  (Physical object for (unnamed))    None   None         {}   
11  (Physical object for (unnamed))    None   None         {}   
12  (Physical object for (unnamed))    None   None         {}   
13  (Physical object for (unnamed))    None   None         {}   
14  (Physical object for (unnamed))    None   None         {}   
15  (Physical object for (unnamed))    None   None         {}   
16  (Physical object for (unnamed))    

In [8]:
spb_hex = gpd.read_file('/home/mvin/PopFrame/examples/data/spb_hex.geojson')
spb_hex

id                                           geometry
0      0  POLYGON ((30.25756 60.03797, 30.27642 60.03224...
1      1  POLYGON ((29.99014 59.84859, 30.00889 59.84288...
2      2  POLYGON ((29.70821 59.86039, 29.72697 59.85473...
3      3  POLYGON ((30.36823 59.89318, 30.38701 59.88741...
4      4  POLYGON ((29.84069 59.99196, 29.85952 59.98629...
..   ...                                                ...
611  611  POLYGON ((30.00787 59.75716, 30.02658 59.75144...
612  612  POLYGON ((30.06399 59.73999, 30.08269 59.73426...
613  613  POLYGON ((29.67125 60.11609, 29.69015 60.11046...
614  614  POLYGON ((29.72764 60.02588, 29.74649 60.02024...
615  615  POLYGON ((29.91588 59.95704, 29.93469 59.95135...

[616 rows x 2 columns]

# РАБОТАЕМ

In [9]:
from popframe.method.engineer import InfrastructureAnalyzer

analyzer = InfrastructureAnalyzer(combined_gdf, spb_hex)
results = analyzer.get_results()
results

id  score    types_in_radius  \
0      0      1  [Энергоснабжение]   
1      1      1  [Энергоснабжение]   
2      2      1  [Энергоснабжение]   
3      3      1  [Энергоснабжение]   
4      4      1  [Энергоснабжение]   
..   ...    ...                ...   
611  611      1  [Энергоснабжение]   
612  612      1  [Энергоснабжение]   
613  613      1  [Энергоснабжение]   
614  614      1  [Энергоснабжение]   
615  615      1  [Энергоснабжение]   

                                              geometry  
0    POLYGON ((30.25756 60.03797, 30.27642 60.03224...  
1    POLYGON ((29.99014 59.84859, 30.00889 59.84288...  
2    POLYGON ((29.70821 59.86039, 29.72697 59.85473...  
3    POLYGON ((30.36823 59.89318, 30.38701 59.88741...  
4    POLYGON ((29.84069 59.99196, 29.85952 59.98629...  
..                                                 ...  
611  POLYGON ((30.00787 59.75716, 30.02658 59.75144...  
612  POLYGON ((30.06399 59.73999, 30.08269 59.73426...  
613  POLYGON ((29.67125 60.11609, 29.69015 60.11046...  
614  POLYGON ((29.72764 60.02588, 29.74649 60.02024...  
615  POLYGON ((29.91588 59.95704, 29.93469 59.95135...  

[616 rows x 4 columns]

In [10]:
results[results['score']>1]

id  score                   types_in_radius  \
173  173      2  [Энергоснабжение, Водоснабжение]   
203  203      2  [Энергоснабжение, Водоснабжение]   
247  247      2  [Энергоснабжение, Водоотведение]   
315  315      2  [Энергоснабжение, Водоснабжение]   
320  320      2  [Энергоснабжение, Водоснабжение]   
456  456      2  [Энергоснабжение, Водоснабжение]   
525  525      2  [Энергоснабжение, Водоотведение]   
546  546      2  [Энергоснабжение, Водоснабжение]   

                                              geometry  
173  POLYGON ((30.26976 59.689, 30.28843 59.68323, ...  
203  POLYGON ((30.13972 59.77832, 30.15844 59.77258...  
247  POLYGON ((30.06498 59.81349, 30.08372 59.80777...  
315  POLYGON ((30.25131 59.70704, 30.27 59.70127, 3...  
320  POLYGON ((30.47995 59.82172, 30.49869 59.81593...  
456  POLYGON ((30.12118 59.79631, 30.13991 59.79057...  
525  POLYGON ((30.02734 59.81268, 30.04608 59.80696...  
546  POLYGON ((30.1021 59.77753, 30.12082 59.7718, ...